<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section5/01_input_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 入力のパディング
入力となる文章の長さは様々なのですが、バッチ内ではデータの時系列の長さを揃える必要があります。  
今回は、短い文章に対して「パディング」を行い、バッチ内の全ての文章の長さを揃えます。

## テンソルのパディング
`nn.utils.rnn.pad_sequence`でパディングを行うことができます。

サイズが異なる3つのテンソルを用意します。

In [ ]:
import torch
import torch.nn as nn

a = torch.ones(3, 5)
print(a)
b = torch.ones(2, 5) * 2
print(b)
c = torch.ones(1, 5) * 3
print(c)

`nn.utils.rnn.pad_sequence`によりパディングを行います。

In [ ]:
padded = nn.utils.rnn.pad_sequence([a, b, c], batch_first=True)
print(padded)

## Tensorのパッキング
このままだと0の入力が多数になってしまうので、RNNで適切に扱うためには「パッキング」を行う必要があります。  
`rnn.pack_padded_sequence`により、0を除いたPackedSequence型のデータを作成できます。  
PackedSequence型のデータは、RNNに入力することが可能です。

In [ ]:
packed = nn.utils.rnn.pack_padded_sequence(padded, [3, 2, 1], batch_first=True, enforce_sorted=False)
print(packed)

## RNNへ入力
パッキングされたTensorをRNNへ入力します。  
出力は`PackedSequence`型のデータに、隠れ層の状態hは通常のTensorになります。

In [ ]:
rnn = nn.RNN(
            input_size=5,  # 入力サイズ
            hidden_size=2,  # ニューロン数
            batch_first=True,  # 入力を (バッチサイズ, 時系列の数, 入力の数) にする
        )

y, h  = rnn(packed)
print(y)
print(h)

## PackedSequence型をTensorに戻す
`pad_packed_sequence`により、RNNの出力をパディングされたTensorに戻します。

In [ ]:
y_unpacked = nn.utils.rnn.pad_packed_sequence(y, batch_first=True)
print(y_unpacked)